In [3]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import pyhepmc
import pandas as pd

In [5]:
# Load the event info
event_info = pd.read_csv("genie_tau_decays_event_info.csv")
particle_info = pd.read_csv("genie_tau_decays_particles.csv")

In [78]:
particle_info.head(13)

,event_num,pdg,E,px,py,pz
0,0,1000080160,14.895081,0.000000,0.000000,0.000000
1,0,-16,3.730323,0.185260,3.295873,1.737299
2,0,-15,3.251366,-0.023894,2.329548,1.409583
3,0,211,0.726490,-0.435732,-0.013640,0.564145
4,0,-16,2.524877,0.411767,2.343313,0.845184
5,1,1000080160,14.895081,0.000000,0.000000,0.000000
6,1,16,3.725654,-1.656092,-2.943322,1.573121
7,1,15,2.846514,-0.845274,-1.491209,1.416815
8,1,-211,0.395154,-0.365821,-0.047146,0.024889
9,1,-211,2.062102,-0.284192,-1.660413,1.181121


In [50]:
# Check if there are any events where two taus are produced
# There should not be any
assert not (particle_info[np.abs(particle_info['pdg']) == 15].groupby("event_num").count() > 1).any().any()
taus = particle_info[np.abs(particle_info["pdg"]) == 15]
assert (taus["event_num"] == np.arange(len(taus))).all()

In [51]:
taus

,event_num,pdg,E,px,py,pz
2,0,-15,3.251366,-0.023894,2.329548,1.409583
7,1,15,2.846514,-0.845274,-1.491209,1.416815
14,2,15,3.332381,-2.127187,-1.737024,-0.636785
19,3,15,3.509529,-0.032933,-0.487587,-2.986784
26,4,15,3.555693,-0.230564,-0.218090,-3.063518
...,...,...,...,...,...,...
447098,76638,-15,1003.199894,-392.660182,-607.363024,-695.222992
447104,76639,15,5031.544231,-4776.946270,-1257.860435,-956.559277
447110,76640,-15,75.985685,56.916040,-50.301859,-0.977036
447116,76641,15,1999.051556,-973.289879,1030.714386,1409.446214


In [98]:
np.unique(np.isclose(particle_info.drop("pdg", axis=1).groupby("event_num").nth[3:].groupby("event_num").sum(), 
           taus.set_index("event_num").drop("pdg", axis=1), rtol=0.01).sum(axis=1), return_counts=True)

(array([0, 1, 2, 3, 4]), array([18011,   568,    61,   493, 57510]))

The number in the first array indicates how many of the 4-momentum components are within the given relative tolerance (rtol), while the number in the second array correspond to how many events have that number of components within the given relative tolerance.

We have a large collection of 4 (all components close in value) and 0 (none of the components are close in value), with a few events in between.

It is good that most events are 4, since that means that the 4-momentum is conserved.

The ones with 0 correspond to when the electron is not in the event, but the electron neutrino and tau neutrino are.

In [24]:
for eventnum, event in particle_info.groupby("event_num"):
    nucleus = event.iloc[0, :]
    nutau = event.iloc[1, :]
    tau = event.iloc[2, :]
    tau_decays = event.iloc[3:, :]
    tau_decay_4m = tau_decays.drop(["event_num", "pdg"], axis=1)
    # print(tau_decay_4m.sum(axis=0))
    tau_4m = tau.drop(["event_num", "pdg"])
    # print(tau_4m)
    if not np.isclose(tau_4m, tau_decay_4m.sum(axis=0), rtol=0.01).all() and set(np.abs(tau_decays["pdg"].unique()).tolist()) != {12, 16}:
        print(tau)
        print(tau_decays)

event_num    32.000000
pdg          15.000000
E             2.754785
px           -1.496317
py            1.480806
pz            0.001224
Name: 197, dtype: float64
     event_num  pdg         E        px        py        pz
198         32  111  0.540061 -0.050864 -0.104457 -0.509851
199         32   16  0.027061 -0.024049  0.005305  0.011216
200         32 -211  2.187703 -1.421406  1.579982  0.499825
event_num    72.000000
pdg          15.000000
E             3.410494
px           -0.018415
py            2.645771
pz            1.214058
Name: 434, dtype: float64
     event_num  pdg         E        px        py        pz
435         72 -211  0.473014  0.062869 -0.178406  0.410464
436         72   16  2.937451 -0.081087  2.824422  0.802922
event_num    332.000000
pdg           15.000000
E              2.880976
px             0.082858
py             2.266286
pz             0.003558
Name: 1971, dtype: float64
      event_num  pdg         E        px        py        pz
1972        332  111